**Table of contents**<a id='toc0_'></a>    
- 1. [Python tips and tricks](#toc1_)    
- 2. [Writing more concise code](#toc2_)    
  - 2.1. [Unpacking iterables](#toc2_1_)    
  - 2.2. [Zipping iterables](#toc2_2_)    
  - 2.3. [Defining sorting order using `key` functions](#toc2_3_)    
  - 2.4. [Anonymous function (aka lambda function)](#toc2_4_)    
- 3. [Making iterations more efficient](#toc3_)    
  - 3.1. [Choosing the right method for iteration](#toc3_1_)    
  - 3.2. [Generator functions](#toc3_2_)    
  - 3.3. [`map()` function](#toc3_3_)    
  - 3.4. [`filter()` function](#toc3_4_)    
- 4. [Reducing memory usage](#toc4_)    
- 5. [Number precision](#toc5_)    
  - 5.1. [Rounding and formatting numbers](#toc5_1_)    
  - 5.2. [Rounding may not work as expected!](#toc5_2_)    
  - 5.3. [Underflow and overflow](#toc5_3_)    
- 6. [Miscellaneous](#toc6_)    
  - 6.1. [Multiplication vs. addition](#toc6_1_)    
  - 6.2. [Using non-built-in data types](#toc6_2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Python tips and tricks](#toc0_)

In [ ]:
import collections
from timeit import default_timer as timer
# from time import time  # timeit is more accurate than time
import numpy as np
import sys

# 2. <a id='toc2_'></a>[Writing more concise code](#toc0_)

- When working with iterables and iterators, you can often write more concise code with functions that work with them directly (no loops needed).
- **iterable**: an object that can be iterated over (e.g. a list, tuple, string)
- **iterator**: an object that generates the next value when you call next() on it
- You can restructure your data easily by **unpacking** and **zipping** iterables (see below)

## 2.1. <a id='toc2_1_'></a>[Unpacking iterables](#toc0_)

- Unpacking: to retrieve individual elements from an iterable
- Use the unpacking operator `*`:

    `a, b, c = *[1, 2, 3]`

In [47]:
a = [1,2,3]
b = [1,1,1]
c = (a, b)

print(c)

([1, 2, 3], [1, 1, 1])


Example use case:

In [48]:
# # Unpacks the tuple 'c' into its individual elements (a and b),
# # and passes them as separate arguments to the function 'func'.

def func(x, y):
    return sum(x) + sum(y) 

print(func(*c))

9


## 2.2. <a id='toc2_2_'></a>[Zipping iterables](#toc0_)

- Zipping: to combine multiple iterables into a single *iterator* of tuples
- Syntax: `zip(iterable1, iterable2, ...)` or `zip(*list_of_iterables)`
- The resulting zip object is an *iterator* of tuples:
    - It generates tuples containing the elements from each iterable at the same index, e.g.:

      `zip([1, 2, 3], [4, 5, 6])` produces an iterator for `[(1, 4), (2, 5), (3, 6)]`

    - length of the zip object = length of the shortest iterable, e.g.:
  
      `zip([1, 2], [4, 5, 6])` produces an iterator for `[(1, 4), (2, 5)]`

- You can convert the zip object to a list or tuple if needed, e.g.:
    - `list(zip([1, 2, 3], [4, 5, 6]))` returns `[(1, 4), (2, 5), (3, 6)]`
    - `list` iterates over the an iterator and collects the results into a list
- Be aware that an iterator **can only be iterated over once!!**
    - If you need to use the zip object multiple times, you need to recreate it

In [49]:
zipped = zip(a, b)
print(zipped)
print(list(zipped))  # convert zipped to a list

zipped = zip(a, b)  # note: zipped was already used above. We need to recreate it
print(tuple(zipped))  # convert zipped to a tuple

[(1, 1), (2, 1), (3, 1)]
((1, 1), (2, 1), (3, 1))


Example use case: Iterate over multiple lists simultaneously

In [50]:
for x, y in zip(a, b):
    print(x, y)

1 1
2 1
3 1


## 2.3. <a id='toc2_3_'></a>[Defining sorting order using `key` functions](#toc0_)

- By default, functions like `sorted()`, `max()`, `min()`, etc. order the elements of an iterable in ascending order.
- You can define a custom sorting order by passing a function as the `key` argument to these functions.
- Example: `sorted(iterable, key=key_function(x))`, where `key_function(x)` is a function that takes an element `x` and returns a value to be used by the outer function.

In [ ]:
# Sort by absolute values
a = [1, -2, 3, -4]
print(sorted(a, key=lambda x: abs(x))) # prints [1, -2, 3, -4]

# Find the element with the maximum absolute value
a = [1, -2, 3, -4]
print(max(a, key=lambda x: abs(x))) # prints -4

# Find the dict key with the maximum dict value
dictionary = {'b': 1, 'c': 2, 'a': 3}
print(max(dictionary)) # prints 'c' (dict keys are sorted alphabetically by default)
print(max(dictionary, key=lambda x: dictionary[x])) # prints 'a' (sort by dict values instead)

[1, -2, 3, -4]
-4
c
a


## 2.4. <a id='toc2_4_'></a>[Anonymous function (aka lambda function)](#toc0_)

- Key functions are often defined as **anonymous functions**
- Syntax: `lambda arguments: expression`
- Only one expression allowed
- It can be used directly or assigned to a variable

In [ ]:
# lambda function with one argument
func = lambda x: x**2
print(func(3))

# lambda function with two arguments
func = lambda x, y: x + y
print(func(1, 2))

9
3


# 3. <a id='toc3_'></a>[Making iterations more efficient](#toc0_)

- There are multiple ways to iterate over data in Python. For example:
    - a loop (e.g. `for` loop)
    - list comprehensions: e.g. `[x for x in iterable]` (This creates a list)
    - generator expressions: e.g. `(x for x in iterable)` (This creates an iterator that generates values on-the-fly)
    - generator functions: e.g. `def my_generator(): yield x` (This creates a function that returns an iterator)
    - `map()` function: apply a function to each element of an iterable
    - `filter()` function: filter elements of an iterable based on a condition
    - `itertools` module: provides functions for creating iterators for efficient looping 
    - Restructure data by unpacking and zipping iterables (see above)

## 3.1. <a id='toc3_1_'></a>[Choosing the right method for iteration](#toc0_)

- Some methods are more efficient than others, depending on the use case, e.g.:
    - Why do you want to iterate over the data? (e.g. to apply a function, filter elements, restructure the data)
    - Do you need to get the results of all iterations at once/ iterate over the entire data right away?
    - Do you need to iterate over the data multiple times or just once?
- Iterating over an iterable (e.g. `for x in [a,b,c,d,e]:`) in one go can be inefficient:
    - It requires loading all items into memory at once
    - The results of all iterations need to be stored in memory, which can be inefficient for large datasets
- If you don't need to iterate over all items right away (e.g. you only need the next item at a time), consider iterating over an iterator instead of an iterable:
    - No need to load all items into memory at once (the next item is generated on-the-fly)
    - You can create an iterator in several ways:
        - Using the `iter()` function on an iterable: `iter(iterable)`
        - Using functions that return iterators, e.g. `range()`, `zip()`, `map()`, `filter()`
        - Using a generator (a special kind of iterator)
            - a **generator expression**: `(x for x in iterable)`
            - a **generator function**: `def my_generator(): yield x`
- If you need to iterate over the same iterator multiple times, consider creating a generator function so you can recreate the iterator easily when needed. (Remember that an iterator can only be iterated over once!)

## 3.2. <a id='toc3_2_'></a>[Generator functions](#toc0_)

- A generator function returns a generator object, which is an iterator
- It uses the `yield` keyword (not `return`) to return values **one at a time**
- Call `next()` on the generator object to get the next value

In [63]:
def my_generator():
    yield 1
    yield 2
    yield 3

# Alternatively:
def my_generator():
    for i in range(1, 4):
        yield i

gen = my_generator()
print(next(gen))  # prints 1
print(next(gen))  # prints 2
print(next(gen))  # prints 3
print(next(gen, 'No more items'))  # prints 'No more items' since the generator is exhausted

1
2
3
No more items


## 3.3. <a id='toc3_3_'></a>[`map()` function](#toc0_)

- The `map()` function applies a function to each element of an iterable (or multiple iterables) and returns a map object (an iterator)
- Syntax: `map(function, iterable)`

In [73]:
def func(x):
    return x**2
a = [1, 2, 3]

print(list(map(func, a)))

# Multiple iterables with map function
def func(x, y):
    return x + y
a = [1, 2, 3]
b = [4, 5, 6]
print(list(map(func, a, b)))

[1, 4, 9]
[5, 7, 9]


`map()` vs list comprehensions:
- `map()` is faster than list comprehensions, but it returns an iterator.
- To get a list, you will need to use `list()`.
    - This means you will still need to iterate over all the items (`list()` iterates over the iterator).
    - So, if have a list to iterate over, you might as well just use list comprehensions.

In [70]:
# Compare the performance of list comprehensions and map function
start = timer()
for i in range(1000000):
    [x**2 for x in a] # returns a list
print(f"Time taken with list comprehension: {timer()-start:.3f} s")

start = timer()
for i in range(1000000):
    map(func, a) # returns a map object
print(f"Time taken with map(func, a): {timer()-start:.3f} s")

start = timer()
for i in range(1000000):
    list(map(func, a)) # returns a list
print(f"Time taken with list(map(func, a)): {timer()-start:.3f} s")


Time taken with list comprehension: 0.401 s
Time taken with map(func, a): 0.204 s
Time taken with list(map(func, a)): 0.608 s


## 3.4. <a id='toc3_4_'></a>[`filter()` function](#toc0_)

- The `filter()` function filters elements of an iterable based on a function and returns a filter object (an iterator)
- Syntax: `filter(function, iterable)`

In [74]:
def func(x): 
    return x % 2 == 0 # to filter even numbers
a = [1,2,3,4,5,6]

print(list(filter(func, a)))

[2, 4, 6]


`filter()` vs `np.where()`:

- `filter()` works with both lists and numpy arrays.
- In some cases, `filter()` is faster than `np.where()` **even for numpy arrays!**

In [79]:
start = time()
for i in range(1000000):
    list(filter(func, a))
print(f"Time taken with list(filter(func, a)): {time()-start:.3f} s")

a = np.array(a)

start = time()
for i in range(1000000):
    list(filter(func, a))
print(f"Time taken with list(filter(func, a)) on numpy array: {time()-start:.3f} s")

start = time()
for i in range(1000000):
    a[np.where(a%2==0)] # get a numpy array using the indices returned by np.where
print(f"Time taken with a[np.where(a%2==0)]: {time()-start:.3f} s")

Time taken with list(filter(func, a)): 2.064 s
Time taken with list(filter(func, a)) on numpy array: 2.116 s
Time taken with a[np.where(a%2==0)]: 5.211 s


# 4. <a id='toc4_'></a>[Reducing memory usage](#toc0_)

- Use **Numpy arrays** instead of lists for numerical data.
- Use **iterators** instead of iterables when you don't need to iterate over all items right away. (See the section on making iterations more efficient)

Example
- Numpy arrays are way more memory-efficient than lists, especially for large datasets:

In [ ]:
a = [[np.random.rand() for _ in range(100)] for _ in range(10000)] # a list of lists (2D array)
a_numpy = np.array(a)

# memory usage (in MB)
a_numpy_size = sys.getsizeof(a_numpy) / (1024*1024)

# Note that sys.getsizeof() returns the size of the outer object, not the size
# of the elements inside it. To get the accurate size of the list 'a', we need to
# sum the sizes of all its elements at all levels (including sublists).
a_size = (sum([sys.getsizeof(x) for sublist in a for x in sublist])
            + sum([sys.getsizeof(sublist) for sublist in a])
            + sys.getsizeof(a)
        ) / (1024 * 1024)
# Alternatively, use pympler to get the size of 'a'
# from pympler import asizeof
# a_size = asizeof.asizeof(a) / (1024 * 1024)

print(f"Memory usage of a: {a_size:.2f} MB")
print(f"Memory usage of a_numpy: {a_numpy_size:.2f} MB")


Memory usage of a: 31.74 MB
Memory usage of a_numpy: 7.63 MB


# 5. <a id='toc5_'></a>[Number precision](#toc0_)

## 5.1. <a id='toc5_1_'></a>[Rounding and formatting numbers](#toc0_)

In [91]:
# --- Rounding numbers ---
print('----- Rounding numbers -----')

# Rounding to n decimal places
# round(number, ndigits)
print(round(1.2304, 3)) # round to 3 decimal places, prints 1.23

# n significant figures (allows trailing zeros)
# f'{variable:format_specifier}'
# OR: format(variable, format_specifier)
print(f'{1.2304:.3f}') # include 3 significant figures, prints 1.230
print(format(1.2304, '.3f')) # include 3 significant figures, prints 1.230

# Rounding to the nearest tenth, hundredth, thousandth, etc.
# round(number, -n)
print(round(12345, -3)) # rounds to the nearest 1000, prints 12000 (2 significant figures)

# --- Scientific notation ---
print('----- Scientific notation -----')

print(f'{1234567890:.2e}') # prints 1.23e+09
# print(format(1234567890, '.2e'))

# Scientific notation with a specific number of significant figures
print(f'{1234567890:.2g}') # prints 1.2e+09 (2 significant figures)
# print(format(1234567890, '.2g'))

----- Rounding numbers -----
1.23
1.230
1.230
12000
----- Scientific notation -----
1.23e+09
1.2e+09


## 5.2. <a id='toc5_2_'></a>[Rounding may not work as expected!](#toc0_)

- Rounding may not work as expected due to floating-point representation errors. For example, 
    - `round(0.1 + 0.2, 2)` does not return `0.3`, but rather `0.30000000000000004`.
    - `round(1.005, 2)` does not return `1.01`, but rather `1.0`.

In [100]:
print(round(0.1 + 0.2, 2))
print(round(1.005, 2))

0.3
1.0


## 5.3. <a id='toc5_3_'></a>[Underflow and overflow](#toc0_)

- Underflow and overflow can occur when performing arithmetic operations that result in values that are too small or too large:
    - **Underflow**: `1e-400` is too small to be represented accurately, resulting in `0.0`.
    - **Overflow**: `1e400` is too large to be represented accurately, resulting in `inf`.
 

In [110]:
print(1e-400) # prints 0.0 (underflow)
print(1e400) # prints inf (overflow)

0.0
inf


In some cases, multiplcation can lead to representation issues, such as underflow (e.g. when multiplying very small numbers together).
- To avoid this, one can take the logarithm of the numbers *before* multiplying them so that each number is replaced by its logarithm and multiplication is replaced by addition of the logarithms. (`log(a * b * c * d) = log(a) + log(b) + log(c) + log(d)`)
- This results in a more stable computation (i.e. less prone to underflow or overflow issues) and allows a wider range of values to be represented.

In [ ]:
a, b, c, d = 2e-100, 2e-100, 2e-100, 2e-100
p, q, r, s = np.log(a), np.log(b), np.log(c), np.log(d)
log_sum = p + q + r + s

print(f"Product of a, b, c, d: {a * b * c * d}")
print(f"Logarithm of product: {log_sum}")

# Note that logrithm of the numbers must be taken before multiplying them
# np.log(a * b * c * d)  # This will return -inf due to underflow

Product of a, b, c, d: 0.0
Logarithm of product: -918.2614484753785


# 6. <a id='toc6_'></a>[Miscellaneous](#toc0_)

## 6.1. <a id='toc6_1_'></a>[Multiplication vs. addition](#toc0_)

- Multiplication is not necessarily slower than addition.
- This is because addition of floating-point numbers can be more complex due to the need to align the decimal points.

In [ ]:
a, b, c, d = 0.25, 0.25, 0.25, 0.25

time_mul = 0
time_add = 0

iter = 1000000
for _ in range(iter):
    # multiplication
    start_mul = timer()
    product = a * b * c * d
    end_mul = timer()
    time_mul += end_mul - start_mul
    # summation
    start_add = timer()
    summed = a + b + c + d
    end_add = timer()
    time_add += end_add - start_add

print(f"Multiplication: {time_mul / iter * 1e6:.3f} microseconds")
print(f"Summation     : {time_add / iter * 1e6:.3f} microseconds")

Multiplication: 0.278 microseconds
Summation     : 0.279 microseconds


## 6.2. <a id='toc6_2_'></a>[Using non-built-in data types](#toc0_)

- The collections module provides alternatives to built-in data types like lists, tuples, and dictionaries. 
- It also provides additional data structures like `deque`, `defaultdict`, `Counter`, and `namedtuple`.

In [ ]:
# The deque data structure
print('----- The deque data structure -----')

# Deque is a double-ended queue that supports adding and removing elements from
# both ends in O(1) time. (A bit faster than lists)
# Syntax: collections.deque(iterable)
# Example: Create a deque object from the list a

a = [1,2,3]
d = collections.deque(a)
print(d) # prints deque([1, 2, 3])

d.append(4) # Add elements to the end
print(d) # prints deque([1, 2, 3, 4])

d.appendleft(0) # Add elements to the beginning
print(d) # prints deque([0, 1, 2, 3, 4])

d.pop() # Remove elements from the end
print(d) # prints deque([0, 1, 2, 3])

d.popleft() # Remove elements from the beginning
print(d) # prints deque([1, 2, 3])

# Notes: deque vs lists
# - deque is faster than lists for adding and removing elements from both ends.
# - deque is much SLOWER than lists for accessing elements in the middle.

# --- list operations --- 

a = [1,2,3]
start = time()
for i in range(1000000):
    a.append(1)
print('list append:', time()-start)

start = time()
for i in range(10000):
    a[500000]
print('list access:', time()-start)

# --- deque operations --- 

a = [1,2,3]
d = collections.deque(a)
start = time()
for i in range(1000000):
    d.append(1)
print('deque append:', time()-start)

start = time()
for i in range(10000):
    d[500000]
print('deque access:', time()-start)



----- The deque data structure -----
deque([1, 2, 3])
deque([1, 2, 3, 4])
deque([0, 1, 2, 3, 4])
deque([0, 1, 2, 3])
deque([1, 2, 3])
list append: 0.17824006080627441
list access: 0.001764535903930664
deque append: 0.1445469856262207
deque access: 1.4988398551940918


In [ ]:
# Counter
print('----- The Counter data structure -----')

# A dictionary that counts the number of occurrences of each element in an
# iterable. It is faster than using a for loop with a dictionary.
# Syntax: collections.Counter(iterable)
# Example: Create a Counter object from the list a

a = [1,2,3,1,2,3,1,2,3]
c = collections.Counter(a)
print(c) # prints Counter({1: 3, 2: 3, 3: 3})



----- The Counter data structure -----
Counter({1: 3, 2: 3, 3: 3})


In [ ]:
# The namedtuple data structure
print('----- The namedtuple data structure -----')

# - A tuple with named fields. Think of it as a dictionary with predefined keys.
# - It is more memory-efficient than using a dictionary.
# Syntax: collections.namedtuple(typename, field_names)
# Example: Create a namedtuple object with the fields x and y

MyClass = collections.namedtuple('MyClass', ['x', 'y']) # MyClass is the typename
p = MyClass(1, 2) # each value is assigned to each of the predefined fields (x, y)
print(p.x, p.y) # prints 1 2
print(p[0], p[1]) # prints 1 2

# namedtuple vs dictionary memory usage
p = MyClass(1, 2)
print('namedtuple memory usage:', p.__sizeof__()) # prints 40
dict = {'x':1, 'y':2}
print('dict memory usage:', dict.__sizeof__()) # prints 216


----- The namedtuple data structure -----
1 2
1 2
namedtuple memory usage: 40
dict memory usage: 168
